In [1]:
import pandas as pd
import numpy as np
import tweepy
import json
import matplotlib.pyplot as plt

In [2]:
import time
from tweepy import OAuthHandler

In [131]:
import datetime

In [3]:
ckey = 'OWea1JDj2RMgv9ktXcKkM3CRI'
csecret = 'zolroSpEUpqFrTABaVHttXDYrKAIWBBGPpFjXZ5j4r201obVWq'
atoken = '927171157018759173-tqtWOcZMkWsnCvUToWsR3W0qSp64jwm'
asecret = 'qMvXFuUpsnZD3b2TB1O49aGLSU6p607hDBnzkaYCdJWxR'

In [4]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [5]:
api = tweepy.API(auth)

### Create Functions to Get Tweets

In [14]:
def get_tweets(query, maxtweets):
    outtweets = [[time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet._json['created_at'],'%a %b %d %H:%M:%S +0000 %Y')),
                  tweet._json['created_at'][0:3], 
                  tweet._json["user"]["screen_name"], 
                  tweet._json["text"],
                  tweet._json["source"].split("\u003e")[1].split("</a\u003c\/a\u003e")[0][:-3], 
                  tweet._json['user']['location'],
                  tweet._json["retweet_count"],
                  tweet._json['favorite_count'],
                  tweet._json['user']['followers_count'],
                  tweet._json['user']['friends_count'],
                  tweet._json['user']['favourites_count']] for tweet in tweepy.Cursor(api.search, q=query,lang = "en", rpp=100).items(maxtweets)]
    return outtweets

In [26]:
def get_all_tweets(queries, maxtweets):
    tweets = []
    for query in queries:
        tweets = tweets + get_tweets(query, maxtweets)
    return tweets

In [125]:
def getDF(tweets):
    columns = ['Created At', 'Day', 'Screen Name', 'Text', 'Source', 'User Location', 'Retweet Count', 'Favourite Count', 'User Followers', 'User Friends', 'User Favorites']
    twDF = pd.DataFrame(tweets, columns=columns)
    twDF['Created At'] = pd.to_datetime(twDF['Created At'])
    twDF['Date Only'] = pd.to_datetime(twDF['Created At'].dt.date)
    return twDF

### Generate Targets

In [64]:
#input user targets (screenames) and keywords 
human_targets = ['bankofengland', 'MarketWatch', 'stockstobuy']
keywords = ['snp500', 'interest rate', 'shares', 'snapchat']

In [65]:
#creates query to extract any tweet containing the keyword by each user specified
def gen_queries(human_targets, keywords):
    user_targets = ['from:' + target for  target in human_targets]
    queries = [word + ', ' + user for word in keywords for user in user_targets]
    return queries

In [66]:
#list of queries
queries = gen_queries(human_targets, keywords)
queries

['snp500, from:bankofengland',
 'snp500, from:MarketWatch',
 'snp500, from:stockstobuy',
 'interest rate, from:bankofengland',
 'interest rate, from:MarketWatch',
 'interest rate, from:stockstobuy',
 'shares, from:bankofengland',
 'shares, from:MarketWatch',
 'shares, from:stockstobuy',
 'snapchat, from:bankofengland',
 'snapchat, from:MarketWatch',
 'snapchat, from:stockstobuy']

In [68]:
tweets = get_all_tweets(queries, 5)
tweets

[['2018-02-26 13:42:02',
  'Mon',
  'MarketWatch',
  'Fed’s Bullard says low U.S. ‘neutral’ interest rate unlikely to rise over next 2 years https://t.co/7PjbjY8HWe',
  'SocialFlow',
  '',
  19,
  11,
  3579666,
  188,
  2638],
 ['2018-02-24 10:01:47',
  'Sat',
  'MarketWatch',
  'The Fed gave little hint in its latest report to Congress that it’s prepared to raise U.S. interest rates more aggr… https://t.co/0nOglv1Jd6',
  'SocialFlow',
  '',
  12,
  13,
  3579666,
  188,
  2638],
 ['2018-02-23 20:18:45',
  'Fri',
  'MarketWatch',
  'Dow jumps 250 points as stock market rises near intraday peaks https://t.co/MWqO5wtO7c',
  'MarketWatch',
  '',
  4,
  11,
  3579666,
  188,
  2638],
 ['2018-02-23 17:27:04',
  'Fri',
  'MarketWatch',
  'U.S. stocks look set to end an up-and-down stretch of trade on a high note https://t.co/gQNW0jUGo2 https://t.co/oA6YO8739o',
  'SocialFlow',
  '',
  7,
  12,
  3579666,
  188,
  2638],
 ['2018-02-23 16:58:17',
  'Fri',
  'MarketWatch',
  'Stocks add to gai

In [126]:
data = getDF(tweets)

### Sentiment Analysis

In [75]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    return analysis.sentiment.polarity

In [127]:
# We create a column with the result of the analysis:
data['SA'] = np.array([analyze_sentiment(tweet) for tweet in data['Text'] ])

# We display the updated dataframe with the new column:
display(data.head(10))

,Created At,Day,Screen Name,Text,Source,User Location,Retweet Count,Favourite Count,User Followers,User Friends,User Favorites,Date Only,SA
0,2018-02-26 13:42:02,Mon,MarketWatch,Fed’s Bullard says low U.S. ‘neutral’ interest...,SocialFlow,,19,11,3579666,188,2638,2018-02-26,-0.166667
1,2018-02-24 10:01:47,Sat,MarketWatch,The Fed gave little hint in its latest report ...,SocialFlow,,12,13,3579666,188,2638,2018-02-24,0.270833
2,2018-02-23 20:18:45,Fri,MarketWatch,Dow jumps 250 points as stock market rises nea...,MarketWatch,,4,11,3579666,188,2638,2018-02-23,0.100000
3,2018-02-23 17:27:04,Fri,MarketWatch,U.S. stocks look set to end an up-and-down str...,SocialFlow,,7,12,3579666,188,2638,2018-02-23,0.002222
4,2018-02-23 16:58:17,Fri,MarketWatch,Stocks add to gains as Fed report eases concer...,MarketWatch,,23,25,3579666,188,2638,2018-02-23,0.000000
5,2018-03-01 15:16:14,Thu,MarketWatch,Overstock shares sink after subsidiary disclos...,MarketWatch,,2,1,3579666,188,2638,2018-03-01,-0.100000
6,2018-02-28 21:19:07,Wed,MarketWatch,Box shares drop more than 10% after earnings h...,MarketWatch,,0,0,3579666,188,2638,2018-02-28,0.500000
7,2018-02-28 19:02:52,Wed,MarketWatch,RT @emilybary: Square shares may not be moving...,TweetDeck,,2,0,3579666,188,2638,2018-02-28,-0.033333
8,2018-02-28 14:01:13,Wed,MarketWatch,Shares of T.J. Maxx and Marshall's parent TJX ...,MarketWatch,,3,5,3579666,188,2638,2018-02-28,0.000000
9,2018-02-28 12:24:12,Wed,MarketWatch,Valeant shares drop 3.5% after Q4 revenue miss...,MarketWatch,,4,5,3579666,188,2638,2018-02-28,0.000000


### Weighting and Grouping

In [128]:
# Calculate weights (simple average of retweet counts and favourite counts)
df= data[['Date Only', 'SA', 'Retweet Count', 'Favourite Count']]
vect = np.array(df["Retweet Count"])
vect2 = np.array(df["Favourite Count"])
weights = np.array((vect+vect2)/2)
SA = np.array(df["SA"])

In [143]:
# Obtain weighted scores
df["weightedSA"] = np.array(SA*weights)

C:\Users\Yi Ying Tan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [144]:
#Obtain aggregate
df["weightedSum"] = df.groupby('Date Only')["weightedSA"].transform('sum')
df2 = df[['Date Only', 'weightedSum']].drop_duplicates()
df2

C:\Users\Yi Ying Tan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date Only,weightedSum
0,2018-02-26,-2.500000
1,2018-02-24,3.385417
2,2018-02-23,2.171111
5,2018-03-01,-0.150000
6,2018-02-28,-0.033333
12,2018-02-22,0.000000


In [145]:
#Mark weekends
df2['day'] = np.array([str(date.weekday()) for date in df2['Date Only']])
df2['iswknd'] = np.array([int(int(day)>=4) for day in df2['day']])
df2
#0 is monday, 6 is sunday

,Date Only,weightedSum,day,iswknd
0,2018-02-26,-2.500000,0,0
1,2018-02-24,3.385417,5,1
2,2018-02-23,2.171111,4,1
5,2018-03-01,-0.150000,3,0
6,2018-02-28,-0.033333,2,0
12,2018-02-22,0.000000,3,0


In [50]:
#hashtags = ['#guns since:2018-02-14', '#maga']

In [67]:
#targets = [hashtags]